In [1]:
!ls -lah

total 2.2G
drwxr-xr-x 3 root root 4.0K Oct 25 18:46 .
drwxr-xr-x 1 root root 4.0K Oct 25 17:15 ..
drwxr-xr-x 2 root root 4.0K Oct 25 18:46 .ipynb_checkpoints
-rw-r--r-- 1 root root  30K Oct 25 18:46 demo.ipynb
-rw-r--r-- 1 root root   72 Oct 25 18:46 demo2.ipynb
-rw-r--r-- 1 root root  11M Jan 12  2023 ghcnd-stations.txt
-rw-r----- 1 root root 2.2G Oct 25 14:17 sf.csv


In [4]:
471859200 / 1024**2 # min needed in MB

450.0

In [5]:
from pyspark.sql import SparkSession
spark = (SparkSession.builder.appName("cs544")
         .master("spark://boss:7077")
         .config("spark.executor.memory", "512M")
         .config("spark.sql.warehouse.dir", "hdfs://nn:9000/user/hive/warehouse")
         .enableHiveSupport()
         .getOrCreate())

23/10/25 18:49:22 WARN SparkContext: Another SparkContext is being constructed (or threw an exception in its constructor). This may indicate an error, since only one SparkContext should be running in this JVM (see SPARK-2243). The other SparkContext was created at:
org.apache.spark.api.java.JavaSparkContext.<init>(JavaSparkContext.scala:58)
java.base/jdk.internal.reflect.NativeConstructorAccessorImpl.newInstance0(Native Method)
java.base/jdk.internal.reflect.NativeConstructorAccessorImpl.newInstance(NativeConstructorAccessorImpl.java:62)
java.base/jdk.internal.reflect.DelegatingConstructorAccessorImpl.newInstance(DelegatingConstructorAccessorImpl.java:45)
java.base/java.lang.reflect.Constructor.newInstance(Constructor.java:490)
py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:247)
py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:374)
py4j.Gateway.invoke(Gateway.java:238)
py4j.commands.ConstructorCommand.invokeConstructor(ConstructorCommand.java:80)
py4j.commands.Con

In [8]:
!hdfs dfs -cp sf.csv hdfs://nn:9000/sf.csv

In [10]:
df = spark.read.format("csv").load("hdfs://nn:9000/sf.csv")

In [12]:
df.limit(3).toPandas()

23/10/25 18:52:10 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


,_c0,_c1,_c2,_c3,_c4,_c5,_c6,_c7,_c8,_c9,...,_c25,_c26,_c27,_c28,_c29,_c30,_c31,_c32,_c33,_c34
0,Call Number,Unit ID,Incident Number,Call Type,Call Date,Watch Date,Received DtTm,Entry DtTm,Dispatch DtTm,Response DtTm,...,Call Type Group,Number of Alarms,Unit Type,Unit sequence in call dispatch,Fire Prevention District,Supervisor District,Neighborhooods - Analysis Boundaries,RowID,case_location,Analysis Neighborhoods
1,221210313,E36,22054955,Outside Fire,05/01/2022,04/30/2022,05/01/2022 02:58:25 AM,05/01/2022 02:59:15 AM,05/01/2022 02:59:25 AM,05/01/2022 03:01:06 AM,...,Fire,1,ENGINE,1,2,5,Hayes Valley,221210313-E36,POINT (-122.42316555403964 37.77781524520032),9
2,220190150,E29,22008871,Alarms,01/19/2022,01/18/2022,01/19/2022 01:42:12 AM,01/19/2022 01:44:13 AM,01/19/2022 01:44:28 AM,01/19/2022 01:46:47 AM,...,Alarm,1,ENGINE,1,3,10,Potrero Hill,220190150-E29,POINT (-122.39469970274361 37.76460987856451),26


In [13]:
df = (spark.read.format("csv")
      .option("header", True)
      .load("hdfs://nn:9000/sf.csv"))

In [14]:
df.limit(3).toPandas()

,Call Number,Unit ID,Incident Number,Call Type,Call Date,Watch Date,Received DtTm,Entry DtTm,Dispatch DtTm,Response DtTm,...,Call Type Group,Number of Alarms,Unit Type,Unit sequence in call dispatch,Fire Prevention District,Supervisor District,Neighborhooods - Analysis Boundaries,RowID,case_location,Analysis Neighborhoods
0,221210313,E36,22054955,Outside Fire,05/01/2022,04/30/2022,05/01/2022 02:58:25 AM,05/01/2022 02:59:15 AM,05/01/2022 02:59:25 AM,05/01/2022 03:01:06 AM,...,Fire,1,ENGINE,1,2,5,Hayes Valley,221210313-E36,POINT (-122.42316555403964 37.77781524520032),9
1,220190150,E29,22008871,Alarms,01/19/2022,01/18/2022,01/19/2022 01:42:12 AM,01/19/2022 01:44:13 AM,01/19/2022 01:44:28 AM,01/19/2022 01:46:47 AM,...,Alarm,1,ENGINE,1,3,10,Potrero Hill,220190150-E29,POINT (-122.39469970274361 37.76460987856451),26
2,211233271,T07,21053032,Alarms,05/03/2021,05/03/2021,05/03/2021 09:28:12 PM,05/03/2021 09:28:12 PM,05/03/2021 09:28:17 PM,05/03/2021 09:29:10 PM,...,Alarm,1,TRUCK,2,2,9,Mission,211233271-T07,POINT (-122.42057572093252 37.76418194637148),20


In [16]:
df = (spark.read.format("csv")
      .option("header", True)
      .option("inferSchema", True)
      .load("hdfs://nn:9000/sf.csv"))

In [17]:
df

DataFrame[Call Number: int, Unit ID: string, Incident Number: int, Call Type: string, Call Date: string, Watch Date: string, Received DtTm: string, Entry DtTm: string, Dispatch DtTm: string, Response DtTm: string, On Scene DtTm: string, Transport DtTm: string, Hospital DtTm: string, Call Final Disposition: string, Available DtTm: string, Address: string, City: string, Zipcode of Incident: int, Battalion: string, Station Area: string, Box: string, Original Priority: string, Priority: string, Final Priority: int, ALS Unit: boolean, Call Type Group: string, Number of Alarms: int, Unit Type: string, Unit sequence in call dispatch: int, Fire Prevention District: string, Supervisor District: string, Neighborhooods - Analysis Boundaries: string, RowID: string, case_location: string, Analysis Neighborhoods: int]

In [18]:
# how to transform the data with functions on columns

In [20]:
from pyspark.sql.functions import expr, col

In [21]:
col("Call Date")

Column<'Call Date'>

In [22]:
expr("Call Date")

Column<'Call AS Date'>

In [24]:
df.select(col("Call Date")).limit(5).toPandas()

,Call Date
0,05/01/2022
1,01/19/2022
2,05/03/2021
3,10/20/2021
4,04/30/2022


In [26]:
df.select(expr("`Call Date`")).limit(5).toPandas()

,Call Date
0,05/01/2022
1,01/19/2022
2,05/03/2021
3,10/20/2021
4,04/30/2022


In [27]:
df.select(expr("`Call Date`").alias("xyz")).limit(5).toPandas()

,xyz
0,05/01/2022
1,01/19/2022
2,05/03/2021
3,10/20/2021
4,04/30/2022


In [31]:
df.select(expr("to_date(`Call Date`, 'MM/dd/yyyy')").alias("Date")).limit(5).toPandas()

,Date
0,2022-05-01
1,2022-01-19
2,2021-05-03
3,2021-10-20
4,2022-04-30


In [ ]:
# GOAL: create a parquet file with this data, with no spaces in the column names

In [38]:
columns = [col(c).alias(c.replace(" ", "_")) for c in df.columns]
columns[:5]

[Column<'Call Number AS Call_Number'>,
 Column<'Unit ID AS Unit_ID'>,
 Column<'Incident Number AS Incident_Number'>,
 Column<'Call Type AS Call_Type'>,
 Column<'Call Date AS Call_Date'>]

In [43]:
df.rdd.getNumPartitions()

17

In [46]:
(df.select(columns)
 .write
 .format("parquet")
 .mode("overwrite")
 .save("hdfs://nn:9000/sf.parquet"))

In [47]:
!hdfs dfs -ls hdfs://nn:9000/

Found 3 items
-rw-r--r--   3 root supergroup   10607756 2023-10-25 18:39 hdfs://nn:9000/ghcnd-stations.txt
-rw-r--r--   3 root supergroup 2265533682 2023-10-25 18:51 hdfs://nn:9000/sf.csv
drwxr-xr-x   - root supergroup          0 2023-10-25 19:05 hdfs://nn:9000/sf.parquet


In [48]:
!hdfs dfs -ls hdfs://nn:9000/sf.parquet

Found 18 items
-rw-r--r--   3 root supergroup          0 2023-10-25 19:05 hdfs://nn:9000/sf.parquet/_SUCCESS
-rw-r--r--   3 root supergroup   27805433 2023-10-25 19:03 hdfs://nn:9000/sf.parquet/part-00000-c0356531-d22a-4fab-b00a-466a1e723e65-c000.snappy.parquet
-rw-r--r--   3 root supergroup   27788944 2023-10-25 19:03 hdfs://nn:9000/sf.parquet/part-00001-c0356531-d22a-4fab-b00a-466a1e723e65-c000.snappy.parquet
-rw-r--r--   3 root supergroup   40477903 2023-10-25 19:03 hdfs://nn:9000/sf.parquet/part-00002-c0356531-d22a-4fab-b00a-466a1e723e65-c000.snappy.parquet
-rw-r--r--   3 root supergroup   36016086 2023-10-25 19:03 hdfs://nn:9000/sf.parquet/part-00003-c0356531-d22a-4fab-b00a-466a1e723e65-c000.snappy.parquet
-rw-r--r--   3 root supergroup   36031567 2023-10-25 19:04 hdfs://nn:9000/sf.parquet/part-00004-c0356531-d22a-4fab-b00a-466a1e723e65-c000.snappy.parquet
-rw-r--r--   3 root supergroup   36080343 2023-10-25 19:04 hdfs://nn:9000/sf.parquet/part-00005-c0356531-d22a-4fab-b00a-466a1e

In [49]:
df = spark.read.format("parquet").load("hdfs://nn:9000/sf.parquet")

In [50]:
df

DataFrame[Call_Number: int, Unit_ID: string, Incident_Number: int, Call_Type: string, Call_Date: string, Watch_Date: string, Received_DtTm: string, Entry_DtTm: string, Dispatch_DtTm: string, Response_DtTm: string, On_Scene_DtTm: string, Transport_DtTm: string, Hospital_DtTm: string, Call_Final_Disposition: string, Available_DtTm: string, Address: string, City: string, Zipcode_of_Incident: int, Battalion: string, Station_Area: string, Box: string, Original_Priority: string, Priority: string, Final_Priority: int, ALS_Unit: boolean, Call_Type_Group: string, Number_of_Alarms: int, Unit_Type: string, Unit_sequence_in_call_dispatch: int, Fire_Prevention_District: string, Supervisor_District: string, Neighborhooods_-_Analysis_Boundaries: string, RowID: string, case_location: string, Analysis_Neighborhoods: int]

In [52]:
df.rdd.getNumPartitions()

6